# Trabalho I Computação gráfica

## Alunos
### Felipe Destaole
### João Pedro Farjoun Silva

In [1]:
# !pip install glfw
# !pip install pyopengl


### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

### Inicializando janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(1920, 1080, "Fundo do mar", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [4]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [12]:
vertices_estrela = []

vertices_estrela = [
    [-0.63, 0.13],[0.22, -0.7],[0.03, 0.13],
    [-0.42, -0.7],[0.42, 0.13],[-0.22, 0.13],
    [-0.31, -0.18],[0.10, -0.18],[-0.10, 0.63]
]

In [13]:
# Número de vértices para o corpo do peixe
num_vertices_circulo = 32
raio = 0.4

vertices_corpo = []

for i in range(num_vertices_circulo):
    angulo = 2 * math.pi * i / num_vertices_circulo
    x = math.cos(angulo) * raio
    y = math.sin(angulo) * raio
    vertices_corpo.append([x, y])

vertices_cauda = [
    [+0.05,  0.0],  # Vértice 1 (ponta da cauda)
    [-0.4,  0.3],  # Vértice 2 (parte superior da cauda)
    [-0.4, -0.3],  # Vértice 3 (parte inferior da cauda)
]



In [14]:
vertice_ceu = []

vertice_ceu = [
    [+2.0, -0.5],
    [+2.0, +0.5],
    [-2.0, -0.5],
    [-2.0, +0.5],
]

In [15]:
num_vertices = 50  # pontos para formar o círculo
num_spikes = num_vertices  # número de espinhos
raio = 0.1
comprimento_espinhos = 0.2
angulo = 0.0

# Listas de vértices
vertices_sol = []
vertices_espinhos = []

# Gera os vértices do círculo
for i in range(num_vertices):
    angulo += 2 * math.pi / num_vertices
    x = math.cos(angulo) * raio
    y = math.sin(angulo) * raio
    vertices_sol.append([x, y])

    # Adiciona vértices dos espinhos
    x_espinho = math.cos(angulo) * (raio + comprimento_espinhos)
    y_espinho = math.sin(angulo) * (raio + comprimento_espinhos)
    vertices_espinhos.append([x, y])
    vertices_espinhos.append([x_espinho, y_espinho])




In [16]:
obj_vert = []


obj_vert.append(vertice_ceu)
obj_vert.append(vertices_espinhos)
obj_vert.append(vertices_corpo)
obj_vert.append(vertices_cauda)
obj_vert.append(vertices_estrela)


obj_vert = np.concatenate(obj_vert)

In [17]:
vertices = np.zeros(obj_vert.shape[0], [("position", np.float32, 2)])

In [18]:
vertices['position'] = obj_vert

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [19]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [20]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [21]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [22]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [23]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [24]:
loc_color = glGetUniformLocation(program, "color")
loc_transform = glGetUniformLocation(program, "mat_transformation")

def draw_element(vertices, color, transform):
    glUniform4f(loc_color, *color)
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, transform)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, len(vertices))

In [25]:
def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [26]:
#### O código dessa célula não está sendo usado.

def key_event(window,key,scancode,action,mods):
    global vel_x, vel_y, angulo, tam, flag, polygon_mode
    max_velo = 0.0003
    
    if key == 265 and vel_y < max_velo : vel_y += 0.00001 #cima
    if key == 264 and vel_y > -max_velo : vel_y -= 0.00001 #baixo
    if key == 263 and vel_x > -max_velo: vel_x -= 0.00001 #esquerda
    if key == 262 and vel_x < max_velo: vel_x += 0.00001 #direita

    if key == 65: angulo += 0.01 # A
    if key == 83: angulo -= 0.01 

    if key == 90: tam += 0.01 # Z
    if key == 88: tam -= 0.01 #X

    # clicar na tecla P
    if key == 80 and action == glfw.PRESS:
        if polygon_mode == GL_FILL:
            polygon_mode = GL_LINE  # Modo wireframe
        else:
            polygon_mode = GL_FILL  # Modo preenchido
    
glfw.set_key_callback(window,key_event)


def mouse_event(window,button,action,mods):
    
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    if button == 0:
        print('cliquei no botao mouse esquerdo...')

glfw.set_mouse_button_callback(window,mouse_event)


In [27]:
def escala(sx,sy,sz):
    return np.array([sx, 0.0, 0.0, 0.0,
                     0.0, sy, 0.0, 0.0,
                     0.0, 0.0, sz, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

In [28]:
def translacao(tx,ty,tz):
    return np.array([1.0, 0.0, 0.0, tx,
                     0.0, 1.0, 0.0, ty,
                     0.0, 0.0, 1.0, tz,
                     0.0, 0.0, 0.0, 1.0], np.float32)

In [29]:
def rotacao(theta):
    return np.array([np.cos(theta), -np.sin(theta), 0.0, 0.0,
                     np.sin(theta), np.cos(theta), 0.0, 0.0,
                     0.0, 0.0, 1.0, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

### Nesse momento, nós exibimos a janela!


In [30]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [31]:
def background():
    glClearColor(0.0, 0.2, 0.5, 1.0)
    glClear(GL_COLOR_BUFFER_BIT) 
    
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #ceu
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(1.0, 1.0, 0))
    glDrawArrays(GL_TRIANGLE_STRIP, 0, len(vertice_ceu))

    glUniform4f(loc_color, 0.94, 0.9, 0.55, 1.0) #areia
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(1.0, -1.4, 0))
    glDrawArrays(GL_TRIANGLE_STRIP, 0, len(vertice_ceu))


    glUniform4f(loc_color, 1.0, 1.0, 0.0, 1.0) #sol
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, translacao(0.9, 0.9, 0))
    glDrawArrays(GL_POLYGON, len(vertice_ceu), len(vertices_espinhos))

In [32]:
def peixe():
    global inicio
    peixe_x = 0.3
    peixe_y = -0.5


    glUniform4f(loc_color, 1.0, 0.5, 0.31, 1.0) #peixe-corpo
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x, peixe_y, 0),escala(0.3,0.3,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) #peixe-olho
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x + 0.07, peixe_y + 0.06, 0),escala(0.04,0.04,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0) #peixe-olho-parte-preta
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x + 0.07, peixe_y + 0.06, 0),escala(0.03,0.03,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_corpo))

    inicio += len(vertices_corpo)

    glUniform4f(loc_color, 1.0, 0.5, 0.31, 1.0) #peixe-cauda
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(peixe_x - 0.05, peixe_y, 0),escala(0.3,0.3,0)))
    glDrawArrays(GL_POLYGON, inicio, len(vertices_cauda))

In [34]:
import math

angulo = 0
vel_x = 0
vel_y = 0
tam = 0.2
s_x = 0
s_y = 0
polygon_mode = GL_FILL

#glEnable(GL_BLEND);
#glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA);


while not glfw.window_should_close(window):

    glfw.poll_events() 

    
    background()

    inicio = len(vertice_ceu) + len(vertices_espinhos)

    peixe_x = 0.3
    peixe_y = -0.5


    peixe()

    inicio += len(vertices_cauda)

    glUniform4f(loc_color, 1.0, 0.84, 0.0, 1.0) #estrela-do-mar-braco-1
    glUniformMatrix4fv(loc_transform, 1, GL_TRUE, multiplica_matriz(translacao(-0.5,-0.7, 0),escala(0.3,0.3,0)))
    glDrawArrays(GL_TRIANGLES, inicio, len(vertices_estrela))
    
    glPolygonMode(GL_FRONT_AND_BACK, polygon_mode)


    
    glfw.swap_buffers(window)

glfw.terminate()

C:\Users\user\anaconda3\Lib\site-packages\glfw\__init__.py:914: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


GLError: GLError(
	err = 1282,
	description = b'opera\xe7\xe3o inv\xe1lida',
	baseOperation = glClearColor,
	cArguments = (0.0, 0.2, 0.5, 1.0)
)

     c = math.cos(angulo)
    s = math.sin(angulo)
 
    s_x += vel_x
    s_y += vel_y

    

    
    
    mat_translation       = np.array([      tam, 0.0, 0.0, s_x, 
                                            0.0, tam, 0.0, s_y, 
                                            0.0, 0.0, 1.0, 0.0, 
                                            0.0, 0.0, 0.0, 1.0], np.float32)

    
    mat_rotation = np.array([       c,   -s,  0.0, s_x-s_x*c + s_y*s, 
                                    s,    c,  0.0, s_y-s_y*c - s_x*s, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    mat_final = multiplica_matriz(mat_rotation, mat_translation)


    # pede a localizacao da variavel mat_transformation no shader
    loc = glGetUniformLocation(program, "mat_transformation")
    
    # set mat_transformation com a mat_rotation
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_final)

    glPolygonMode(GL_FRONT_AND_BACK, polygon_mode)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos